In [ ]:
import numpy as np
import pandas as pd
import cmath
from sklearn.preprocessing import MinMaxScaler
import scipy.io

### import data

In [ ]:
trainfile = '16QAM_8Pilots_Train.mat'

bps = scipy.io.loadmat(trainfile)['bps']
M = scipy.io.loadmat(trainfile)['M']
nFrames = scipy.io.loadmat(trainfile)['nFrames']
nSymbols = scipy.io.loadmat(trainfile)['nSymbols']
nPilots = scipy.io.loadmat(trainfile)['nPilots']
nCP = scipy.io.loadmat(trainfile)['nCP']
SNR = scipy.io.loadmat(trainfile)['SNR']
clipping = scipy.io.loadmat(trainfile)['clipping']

train_sample = scipy.io.loadmat(trainfile)['input_samples']
train_label = scipy.io.loadmat(trainfile)['input_labels']

In [ ]:
train_sample

In [4]:
train_label

array([[0, 1, 1, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 1],
       [0, 1, 0, ..., 0, 1, 1],
       ...,
       [0, 1, 0, ..., 1, 1, 0],
       [0, 1, 0, ..., 0, 0, 1],
       [1, 0, 0, ..., 1, 0, 1]], dtype=uint8)

In [ ]:
train_sample.shape

In [ ]:
train_label.shape

In [ ]:
train_label = train_label[:,0:8*bps] # get 8 symbols
train_label.shape

In [8]:
from sklearn.utils import shuffle
train_label, train_sample = shuffle(train_label, train_sample)

### build dnn model

In [15]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, BatchNormalization, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import L1
from tensorflow.keras.callbacks import ReduceLROnPlateau

In [31]:
#DNN
model = Sequential([
    BatchNormalization(input_shape=(nSymbols*2*2,)),
    Dense(units=500, activation='relu'),
    BatchNormalization(),
    Dense(units=250, activation='relu'),
    BatchNormalization(),
    Dense(units=120, activation='relu'),
    BatchNormalization(),
    Dense(units=8*bps, activation='sigmoid')
])

In [ ]:
#RNN
model = Sequential([
    BatchNormalization(input_shape=(timestep, (nSymbols*2*2)//timestep)),
    SimpleRNN(units=500, activation='relu', return_sequences=True),
    BatchNormalization(),
    SimpleRNN(units=250, activation='relu', return_sequences=True),
    BatchNormalization(),
    SimpleRNN(units=120, activation='relu'),
    BatchNormalization(),
    Dense(units=n_output, activation='sigmoid')
])

In [11]:
def bit_err(y_true, y_pred):
    err = 1 - tf.reduce_mean(
        tf.reduce_mean(
            tf.cast(
                tf.equal(
                    tf.sign(
                        y_pred - 0.5),
                    tf.cast(
                        tf.sign(
                            y_true - 0.5),
                        tf.float32)), tf.float32),
            1))
    return err

In [12]:
initial_learning_rate = 0.001
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=5000,
    decay_rate=0.85,
    staircase=True)

In [27]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.96,
                              patience=10, min_lr=0.00001)

In [32]:
model.compile(Adam(learning_rate=lr_schedule), loss='mse', metrics=[bit_err])
checkpoint = callbacks.ModelCheckpoint('./temp_trained_25.h5', monitor='val_bit_err',
                                       verbose=0, save_best_only=True, mode='min', save_weights_only=True)

In [ ]:
model.fit(x=train_sample, y=train_label, validation_split=0.1, batch_size=1000, epochs=10000, shuffle=True, verbose=2, callbacks=[checkpoint])

### Test the model

In [ ]:
testfile = '16QAM_8Pilots_Test.mat'
test_sample = scipy.io.loadmat(testfile)['input_samples']
test_label = scipy.io.loadmat(testfile)['input_labels']

In [ ]:
# test
model.load_weights('./temp_trained_25.h5')
BER = []
for SNR in range(5, 30, 5):
    y = model.evaluate(
        test_sample, test_label
    )
    BER.append(y[1])
    print(y)
print(BER)
BER_matlab = np.array(BER)
import scipy.io as sio
sio.savemat('BER_16QAM_K64_8output_8pilot.mat', {'BER':BER_matlab})